## Importando as bibliotecas necessárias

Nesse notebook as bibliotecas utilizadas serão:

* Tensorflow - Para criação e treinamento das redes neurais e outras funções de pré-processamento de dados
* Numpy - Para funções matemáticas

E outras bibliotecas nativas do Python - time e os

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
arquivo_path = "machado.txt"

In [ ]:
text = open(arquivo_path, 'rb').read().decode(encoding='utf-8')
print(f'O texto contém: {len(text)} caracteres')

vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

## Pré-processando os dados

Para a nossa rede, cada caracter precisa ser convertido em um valor numérico para identificação, uma vez que as redes neurais só trabalham com números.

In [ ]:
caracteres_para_ids = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids_para_caracteres = tf.keras.layers.StringLookup(
    vocabulary=caracteres_para_ids.get_vocabulary(), invert=True, mask_token=None)


In [ ]:
# Função para converter de volta os IDs em textos
def text_from_ids(ids):
  return tf.strings.reduce_join(caracteres_para_ids(ids), axis=-1)

## Dividindo o texto em sequencias de entrada

Em seguida, precisamos dividir o texto em sequencias menores de entrada para que a nossa rede possa receber no treinamento


In [ ]:
texto_em_ids = caracteres_para_ids(tf.strings.unicode_split(text, 'UTF-8'))
dataset_de_ids = tf.data.Dataset.from_tensor_slices(texto_em_ids)

In [ ]:
tamanho_da_sequencia_em_caracteres = 100
dados_por_dataset = len(text) // (tamanho_da_sequencia_em_caracteres + 1)

In [ ]:
sequencias = dataset_de_ids.batch(tamanho_da_sequencia_em_caracteres+1, drop_remainder=True)

In [ ]:
def divide_input(sequencia):
    input_text = sequencia[:-1]
    target_text = sequencia[1:]
    return input_text, target_text

In [ ]:
dataset = sequencias.map(divide_input)

## Criando os lotes de treinamento
Utilizando funções do Tensorflow, vamos converter os dados em um formato de entrada válido para a nossa rede neural

In [ ]:
TAMANHO_BATCH = 64

# Tamanho do buffer para embaralhar os exemplos do dataset
TAMANHO_BUFFER = 10000

dataset = (
    dataset
    .shuffle(TAMANHO_BUFFER)
    .batch(TAMANHO_BATCH, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))



## Construindo o modelo
Utilizando o `keras.model`, vamos criar uma rede neural recorrente com duas camadas LSTM (Long short-term memory).
No artigo original, o autor cria um modelo muito semelhante com uma única camada GRU (Gated Recurrent Unit), que são mais baratas computacionalmente porém possuem uma menor eficiencia devido a sua arquitetura com 2 portões (atualização e redefinição), ao contrário da LSTM que possui 3 portões (entrada, esquecimento e saída).

A arquitetura da rede é bem simples:
- Uma camada embedding com a dimensão de 256
- Duas camadas LSTM com 2048 neuronios
- Uma camada dense com o mesmo tamanho que a quantidade de caracteres, onde cada neurônio representa um caracter

O uso de duas camadas LSTM garante que o modelo aprenda padrões de maneira eficiente e com maior precisão

In [ ]:
dimensao_de_embedding = 256
neuronios_lstm = 2048

In [ ]:
class ModeloDeLinguagem(tf.keras.Model):
  def __init__(self, tamanho_do_vocabulario, dimensao_de_embedding, neuronios_lstm):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(tamanho_do_vocabulario, dimensao_de_embedding)
    self.lstm1 = tf.keras.layers.LSTM(neuronios_lstm,
                                      return_sequences=True,
                                      return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(neuronios_lstm,
                                      return_sequences=True,
                                      return_state=True)
    self.dense = tf.keras.layers.Dense(tamanho_do_vocabulario)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = self.embedding(inputs, training=training)
    batch_size = tf.shape(inputs)[0]

    if states is None:
        h0_1 = tf.zeros((batch_size, self.lstm1.units))
        c0_1 = tf.zeros((batch_size, self.lstm1.units))
        x, h1, c1 = self.lstm1(x, initial_state=[h0_1, c0_1], training=training)

        h0_2 = tf.zeros((batch_size, self.lstm2.units))
        c0_2 = tf.zeros((batch_size, self.lstm2.units))
        x, h2, c2 = self.lstm2(x, initial_state=[h0_2, c0_2], training=training)
    else:
        (h1, c1), (h2, c2) = states
        x, h1, c1 = self.lstm1(x, initial_state=[h1, c1], training=training)
        x, h2, c2 = self.lstm2(x, initial_state=[h2, c2], training=training)

    x = self.dense(x, training=training)

    if return_state:
        return x, [(h1, c1), (h2, c2)]
    else:
        return x

In [ ]:
model = ModeloDeLinguagem(
    tamanho_do_vocabulario=len(caracteres_para_ids.get_vocabulary()),
    dimensao_de_embedding=dimensao_de_embedding,
    neuronios_lstm=neuronios_lstm)

## Treinando o modelo!!!!


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Salva os pesos do modelo em checkpoints do treinamento

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])